# **Basic Folding Algorithm using PyRosetta**

Reference: https://rosettacommons.github.io/PyRosetta.notebooks/

In [ ]:
# Install pyrosetta. This will take few minutes
# You need to log in with your Google account.
# You need to allow access to your google drive, to run this script.

!pip install pyrosettacolabsetup
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()
import pyrosetta; pyrosetta.init()
from pyrosetta import *
from pyrosetta.teaching import *
init()

In [ ]:
### Create a initial pose
polyA = pyrosetta.pose_from_sequence('A' * 10)
polyA.pdb_info().name("polyA")

print("phi: %i" %polyA.phi(9))
print("psi: %i" %polyA.psi(9))

polyA.sequence()


# Step 1: Random Move

For the random trial move, write a subroutine to choose one residue at random using random.randint() and then randomly perturb either the φ or ψ angles by a random number chosen from a Gaussian distribution. Use the Python built-in function random.gauss() from the random library with a mean of the current angle and a standard deviation for a given amount. After changing the torsion angle, use pmm.apply(polyA) to update the structure in PyMOL.

In [ ]:
import math
import random

def randTrial(your_pose, move_size):
    randNum = random.randint(2, your_pose.total_residue()) # Picking up a random residue index
    currPhi = your_pose.phi(randNum)
    currPsi = your_pose.psi(randNum)
    newPhi = random.gauss(currPhi, move_size)
    newPsi = random.gauss(currPsi, move_size)
    your_pose.set_phi(randNum,newPhi)
    your_pose.set_psi(randNum,newPsi)
    return your_pose

# Step 2: Scoring Move
For the scoring step, we need to create a scoring function and make a subroutine that simply returns the numerical energy score of the pose.

In [ ]:
sfxn = get_fa_scorefxn()

def score(your_pose):
    return sfxn(your_pose)

# Step 3: Accepting/Rejecting Move
For the decision step, we need to make a subroutine that either accepts or rejects the new conformatuon based on the Metropolis criterion. The Metropolis criterion has a probability of accepting a move as  𝑃=exp(−Δ𝐺/𝑘𝑇) . When  Δ𝐸≥0 , the Metropolis criterion probability of accepting the move is  𝑃=exp(−Δ𝐺/𝑘𝑇) . When  Δ𝐸<0 , the Metropolis criterion probability of accepting the move is  𝑃=1 . Use  𝑘𝑇 in  Rosetta Energy Unit (REU).

In [ ]:
def decision(before_pose, after_pose):
    E = score(after_pose) - score(before_pose)
    kT = 0.4
    if E < 0:
        return after_pose
    elif random.uniform(0, 1) >= math.exp(-E/kT):
        return before_pose
    else:
        return after_pose

# Step 4: Execution
Now we can put these three subroutines together in our main program! Write a loop in the main program so that it performs iterations of: making a random trial move, scoring the protein, and accepting/rejecting the move.

After each iteration of the search, output the current pose and energy.

In [ ]:
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Create a folder in the root directory to save pdbs
!mkdir -p "/content/drive/MyDrive/colab_folding/"

### BEGIN MC folding
your_pose = Pose() # Create an empty pose
your_pose.assign(polyA)

lowest_pose = Pose()

score_total = []
for i in range(100000):

    before_pose = Pose()
    before_pose.assign(your_pose) # keep track of pose before random move
    after_pose = Pose()

    if score(your_pose) > 5:
      move_size = 30
    elif score(your_pose) > 0:
      move_size = 25
    else:
      move_size = 20

    after_pose.assign(randTrial(your_pose, move_size)) # do random move and store the pose
    your_pose.assign(decision(before_pose, after_pose)) # keep the new pose or old pose

    if score(your_pose) < score(lowest_pose): # updating lowest pose
      lowest_pose.assign(your_pose)

    if i%1000 == 0:
      print("Iteration # %i" %i) # output
      print("Current pose score: %1.3f" %score(your_pose)) # output
      your_pose.dump_pdb('/content/drive/My Drive/colab_folding/folding_%04d.pdb'%(i))
    score_total.append(score(your_pose))

lowest_pose.dump_pdb('/content/drive/My Drive/colab_folding/best_folding.pdb')


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(score_total)
plt.ylabel('Score',fontsize=15)
plt.xlabel('MC steps',fontsize=15)
